In [ ]:
#i "nuget:https://api.nuget.org/v3/index.json"
#r "nuget: Microsoft.Data.Analysis, 0.19.0"
#r "nuget: Plotly.NET, 2.0.0-preview.8"
#r "nuget: Plotly.NET.Interactive, 2.0.0-preview.8"
#r "nuget: Deedle"
#r "nuget: FSharp.Data"
#r "nuget: FSharp.Core, 5.0.0"
#r "nuget: CsvHelper"

Restore sources https://api.nuget.org/v3/index.json Installed Packages CsvHelper, 27.2.0 Deedle, 2.5.0 FSharp.Core, 5.0.0 FSharp.Data, 4.2.5 Microsoft.Data.Analysis, 0.19.0 Plotly.NET, 2.0.0-preview.8 Plotly.NET.Interactive, 2.0.0-preview.8

In [ ]:
using Microsoft.Data.Analysis;
using System.Linq;
using Microsoft.AspNetCore.Html;
using System.IO;
using Deedle;
using System;
using Plotly.NET;
using Plotly.NET.LayoutObjects;
using Microsoft.FSharp.Core;
using System.Text.Json;

In [ ]:
public const string BasePath ="C:\\DCUGoogleDrive\\DataVisualisationAssignmentData\\";
public static DataFrame ReadCSVToDataFrame(string filePath, int guessRows = 10){
    DataFrame df;
    using (FileStream stream = File.OpenRead(filePath)){
        df = DataFrame.LoadCsv(stream, numberOfRowsToRead:-1, guessRows:guessRows );
    }
return df;
}

In [ ]:
var fullFrame = ReadCSVToDataFrame($"{BasePath}DataVisualisationScrapedFinal.csv");

In [ ]:
fullFrame.Rows.Count()

25847

In [ ]:
fullFrame.Columns.Remove("Unnamed: 0");
fullFrame.Columns.Remove("Unnamed: 0.1");
fullFrame.Columns.Remove("Unnamed: 0.1.1");
fullFrame.Columns.Remove("Unnamed: 0.1.1.1");
fullFrame.Columns.Remove("Unnamed: 0.1.1.1.1");
fullFrame.Columns.Remove("Unnamed: 0.1.1.1.1.1");
fullFrame.Columns.Remove("Unnamed: 0.1.1.1.1.1.1");
fullFrame

In [ ]:
public static void PrintNullRate(DataFrame frame){
    Console.WriteLine(frame.Info());
    foreach(var col in frame.Columns){
        decimal noOfNulls;
        if(col.DataType == typeof(string))
    noOfNulls = col.OfType<string>().Count(_ => string.IsNullOrEmpty(_));
    else 
    noOfNulls = col.NullCount;
    
        decimal null_rate = (decimal)(noOfNulls / frame.Rows.Count) * 100.00m; 
        Console.WriteLine(string.Format("{0}'s null rate :{1}%", col.Name , null_rate));
    }
}

In [ ]:
Console.WriteLine("All");
PrintNullRate(fullFrame);

All
Info                                    source              title               type                director            cast                production_countriesdate_added          release_year        rating              duration            genres              description         audience            
DataType            System.Single       System.String       System.String       System.String       System.String       System.String       System.String       System.DateTime     System.Single       System.String       System.String       System.String       System.String       System.String       
Length (excluding null values)25847               25847               25847               25847               25847               25847               25847               25505               25847               25847               25847               25847               25847               25847               

's null rate :0.00%
source's null rate :0.00%
title's null rate :0.00%
type's null

In [ ]:
Console.WriteLine($"Total Number of Records: {fullFrame.Rows.Count()}");
Console.WriteLine($"Netflix Records: {fullFrame.Filter(fullFrame["source"].ElementwiseEquals("netflix")).Rows.Count()}");
Console.WriteLine($"Amazon Records: {fullFrame.Filter(fullFrame["source"].ElementwiseEquals("amazon-prime-video")).Rows.Count()}");
Console.WriteLine($"Disney Records: {fullFrame.Filter(fullFrame["source"].ElementwiseEquals("disney-plus")).Rows.Count()}");
Console.WriteLine($"HBO Records: {fullFrame.Filter(fullFrame["source"].ElementwiseEquals("hbo-max")).Rows.Count()}");
Console.WriteLine($"Hulu Records: {fullFrame.Filter(fullFrame["source"].ElementwiseEquals("hulu")).Rows.Count()}");

Total Number of Records: 25847
Netflix Records: 6054
Amazon Records: 10489
Disney Records: 1457
HBO Records: 4740
Hulu Records: 3107


## About the data

Netflix and Disney's data is in good nick, it doesn't require much work, 
Hulu is missing a lot of data around the Cast, Director and Country.
And Amazon is missing alot of data for when it was added.

Amazon won't be usable in year on year growth at this rate.

In [ ]:
public class Row : ICloneable 
{
    public Row(){}
    public Row(string source, DataFrameRow _){
        Source = source;
        Id = _[0].ToString();
        Type = _[3].ToString();
            Title = _[2].ToString();
            Director = _[4].ToString();
            Cast = _[5].ToString();
            Country = _[6].ToString();
            DateAdded = DateTime.Parse(_[7].ToString());
            ReleaseYear = int.Parse(_[8].ToString());	
            Rating = _[9].ToString();
            Duration = _[10];
            Genre = _[11].ToString();
            Description = _[12].ToString();
            Audience = _[13].ToString();
    }
public string Source {get; set;}
public string Id {get; set;}
public string Type {get; set;}
public string Title {get; set;}
public string Director {get; set;}
public string Cast {get; set;}
public string Country {get; set;}
public DateTime DateAdded {get; set;}
public int? ReleaseYear {get; set;}
public string Rating {get; set;}
public object Duration  {get; set;}
public string Genre {get; set;}
public string Description {get; set;}
public string Audience {get; set;}

public object Clone()
{
    return this.MemberwiseClone();
}
}

In [ ]:
public static string ToTitle(this string input){
    switch(input) {
        case "netflix": 
        return "Netflix"; 
        case "amazon-prime-video":
        return "Amazon";
        case "disney-plus":
        return "Disney Plus";
        case "hulu":
        return "Hulu";
        case "hbo-max":
        return "HBO";
        default:
        return "Unknown";
    }
}

## Dropping Stuff

Right off the bat I'm going to drop out the missing rows for date added and rating. Its a small amount so its not going to make a diff

In [ ]:
List<Row> allData = new();

foreach(var row in fullFrame.Rows){
try{
    allData.Add(new Row(row[1].ToString(), row));
}
catch{

}
}
allData

index,Source,Id,Type,Title,Director,Cast,Country,DateAdded,ReleaseYear,Rating,Duration,Genre,Description,Audience
0,netflix,0,movie,Kannathil Muthamittal,Mani Ratnam,"Madhavan, Simran, P.S. Keerthana, Prakash Raj, Nandita Das, Delhi Kumar, J.D. Chakravarthi",India,2021-11-17 00:00:00Z,2002,8.4/10,130 min,"Dramas, International Movies","In this powerful tale about abandonment and reunion, a 9-year-old's blissful world collides with reality when she learns she was adopted as an infant.",TV-14
1,netflix,1,movie,Mokalik (Mechanic),Kunle Afolayan,"Tooni Afolayan, Ayo Ogunshina, Dayo Akinpelu, Simi Ogunleye, Femi Adebayo, Lateef Adedimeji, Damilola Ogunsi, Rasaq Olayiwola, Ayo Adesanya, Wale Akorede, Toyosi Benjamin, Hamzat Sherifdeen, Samuel Olasehinde",Nigeria,2021-11-17 00:00:00Z,2019,6.4/10,100 min,"Dramas, International Movies",A young boy's life shifts when he's sent to work at a mechanic's shop far from his suburban home until an opportunity forces him to make a tough choice.,TV-14
2,netflix,2,movie,My Dad's Christmas Date,Mick Davis,"Jeremy Piven, Olivia Mai Barrett, Hadar Cats, Joely Richardson, Megan Brown Martinez, Felix Butterwick",United Kingdom,2021-11-17 00:00:00Z,2020,5.5/10,91 min,"Comedies, Dramas",A savvy 16-year-old attempts to play matchmaker among the mistletoe when she secretly sets up online dates for her widowed dad during the holidays.,TV-MA
3,netflix,3,movie,Nenjamundu Nermaiyundu Odu Raja,Karthik Venugopalan,"Rio Raj, R. J. Vignesh, Shirin Kanchwala",India,2021-11-17 00:00:00Z,2019,6.2/10,117 min,"Comedies, Dramas, International Movies","Offered a large sum of money to complete three challenging tasks, two carefree content creators are in for an entertaining — and eye-opening — journey.",TV-14
4,netflix,4,movie,October 1,Kunle Afolayan,"Sadiq Daba, David Bailie, Kayode Olaiya, Kehinde Bankole, Fabian Adeoye Lojede, Nick Rhys, Kunle Afolayan, Colin David Reese, Ibrahim Shatta, Femi Adebayo, Kanayo O. Kanayo, Lawrence Stubbings, Ademola Adedoyin",Nigeria,2021-11-17 00:00:00Z,2014,6.4/10,149 min,"Dramas, International Movies, Thrillers","Against the backdrop of Nigeria's looming independence from Britain, detective Danladi Waziri races to capture a killer terrorizing local women.",TV-14
5,netflix,5,movie,Phone Swap,Kunle Afolayan,"Wale Ojo, Nse Ikpe-Etim, Lydia Forson, Joke Silva, Chika Okpala, Ada Ameh, Hafeez Oyetoro, Chika Chukwu",Nigeria,2021-11-17 00:00:00Z,2012,6.0/10,112 min,"Comedies, International Movies, Romantic Movies",A fashion designer and an arrogant businessman reluctantly try to help each other after accidentally swapping phones in the airport.,TV-14
6,netflix,6,movie,Prayers for the Stolen,,"Ana Cristina Ordóñez González, Blanca Itzel Pérez, Camila Gaal, Marya Membreño, Giselle Barrera Sánchez, Alejandra Camacho, Mayra Batalla, José Estrada, Julián Guzmán Girón, Norma Pablo, Memo Villegas, Teresa Sánchez",,2021-11-17 00:00:00Z,2021,,111 min,"Dramas, Independent Movies, International Movies","In a mountainous region of Mexico where poppies abound, three girls take refuge in their friendship to cope with the trials brought on by a drug cartel.",R
7,netflix,7,tv-show,Tear Along the Dotted Line,,"Zerocalcare, Valerio Mastandrea, Paolo Vivio, Chiara Gioncardi, Veronica Puccio",,2021-11-17 00:00:00Z,2021,,1 Season,"International TV Shows, TV Comedies",A cartoonist in Rome with his armadillo-for-a-conscience reflects on his path in life and a would-be love as he and his friends travel outside the city.,TV-MA
8,netflix,8,movie,The Bridge,Kunle Afolayan,"Chidinma Ekile, Ademola Adedoyin, Kunle Afolayan, Zack Orji, Tina Mba, Ayo Akinwale, Akim Mogaji, Ayo Mogaji, Ken Erics",Nigeria,2021-11-17 00:00:00Z,2017,5.2/10,119 min,"Dramas, International Movies, Romantic Movies",A Yoruba prince and a young lady from a prominent Igbo family face tribal prejudice and parental pressure when they secretly wed.,TV-14
9,netflix,9,movie,The CEO,Kunle Afolayan,"Wale Ojo, Jimmy Jean-Louis, Hilda Dokubo, Nico Panagio, A

## Get the data into friendly shapes

1. Source, Number of Tv Shows, Number of Movies
2. Content Added by Year so , Source Number added in a month
3. Genres
4. Production by country, if I want to do this apply similiar approach as genre, but different filters

In [ ]:
var sourceAndTotalContentType =allData.GroupBy(_ => _.Source).Select(_ => new {Source = _.Key, Movies = _.Count(_ => _.Type == "movie"), TVShows = _.Count(_ => _.Type == "tv-show")});

In [ ]:
var sourceAndMonthAddedCounts = allData.GroupBy(_ => new {Source = _.Source, MonthAdded = $"{_.DateAdded.Year}/{_.DateAdded.Month}/1"}).Select(_ => new {Source = _.Key.Source, MonthAdded = _.Key.MonthAdded, Count = _.Count()});

In [ ]:
public class GenreRow {
public string Genre {get; set;}
public Row Row {get; set;}

public Row ToRow(){

    var newRow = (Row)Row.Clone();
    newRow.Genre = Genre;
    return newRow;
    }
}

# Genres

These are a bit of a mess so gonna do some hand tailored swaps and combinations, and not going to make a distinction between movie and tv show.
Some example, Sports and Sports Movies will be combined into Sports, 

In [ ]:
public static string Remove(this string source, string target) => source.Replace(target, "");

In [ ]:
public static string ProcessString(this string source){
    return  source.Replace(" and ", " & ")
    .Remove("Movies")
    .Remove(" TV Shows")
    .Replace("TV Shows", "TV Show")
    .Replace("TV Show", "placeholder1")
    .Remove("TV")
    .Replace("placeholder1", "TV Show")
    .Replace("Comedies", "Comedy")
    .Replace("Game Shows", "Game Show / Competition")
    .Replace("Kids'", "Kids")
    .Replace("Kids & Family", "Kids")
    .Replace("Children & Family", "Kids")
    .Replace("LGBTQ+", "LGBTQ")
    .Replace("LGBTQ", "LGBTQ+")
    .Remove("& Culture")
    .Replace("Historical", "History")
    .Remove("Series")
    .Replace("ies", "y")
    .Replace("Science Fiction", "Sci-Fi & Fantasy")
    .Remove("& Variety")
    .Replace("Thrillers", "Thriller")
    .Replace("Anime Features", "Anime")
    .Replace("Action-Adventure", "Action & Adventure")
    .Replace("Classics", "Classic")
    .Replace("Docusery", "Docuseries")
    .Replace("Dramas", "Drama")
    .Replace("/", " & ")
    .Replace("Talk Shows", "Talk Show")
    .Replace("Romantic", "Romance")
    .Remove("Videos")
    .Trim();
    }
//File.WriteAllLines("C:\\DCUGoogleDrive\\DataVisualisationAssignmentData\\Genres.csv", allData.SelectMany(_ => _.Genre.Split(",")).Select(_ => _.Trim()).Distinct().Select(x => string.Join(",", x)));
File.WriteAllLines("C:\\DCUGoogleDrive\\DataVisualisationAssignmentData\\Genres.csv", allData.SelectMany(_ => _.Genre.Split(",")).Select(_ => _.ProcessString()).SelectMany(_ => _.Split(" & ")).Select(_ => _.Trim()).Distinct().Select(x => string.Join(",", x)));

allData.SelectMany(_ => _.Genre.Split(",")).Select(_ => _.ProcessString()).SelectMany(_ => _.Split(" & ")).Select(_ => _.Trim())
.Distinct()

index,value
0,Drama
1,International
2,Comedy
3,Thriller
4,Romance
5,Independent
6,Action
7,Adventure
8,Horror
9,Crime


In [ ]:
var splitBySourceAndGenre = allData.SelectMany(_ => _.Genre.Split(",").Select(_ => _.ProcessString()).SelectMany(_ => _.Split(" & ")).Select(_ => _.Trim())
.Distinct().Select(s => 
    new GenreRow(){
        Genre = s, 
        Row =_
    })).Select(_ => _.ToRow())
    .GroupBy(_ => new {Source = _.Source, Genre =_.Genre})
    .Select(_ => new {Source = _.Key.Source, Genre = _.Key.Genre, Amount = _.Count()});

In [ ]:
File.WriteAllLines("C:\\DCUGoogleDrive\\DataVisualisationAssignmentData\\countries.csv", allData.Where(_ => !string.IsNullOrEmpty(_.Country)).SelectMany(_ => _.Country.Split(",")).Select(_ => _).Select(_ => _.Trim())
.Distinct().Select(x => string.Join(",", x)));

allData.Where(_ => !string.IsNullOrEmpty(_.Country)).SelectMany(_ => _.Country.Split(",")).Select(_ => _.Trim())
.Distinct();

var countryCount = allData.Where(_ => !string.IsNullOrEmpty(_.Country)).SelectMany(_ => _.Country.Split(",")).Select(_ => _.Trim()).GroupBy(_ => _).Select(_=> new {Country = _.Key, Amount = _.Count()});
//11517 records with null country, do I care? No, just don't split by source

In [ ]:
using CsvHelper;
using System.IO;
using System.Globalization;
using System.Text.Json;

public void WriteToCSV<T>(string filePath, IEnumerable<T> records){
    using (var writer = new StreamWriter(filePath))
    using (var csv = new CsvWriter(writer, CultureInfo.InvariantCulture))
    {
        csv.WriteRecords(records);
    }
}

public void WriteToJsonFile<T>(string filePath, T records){
    string json = JsonSerializer.Serialize(records);
    var stream = new MemoryStream();
    using (var writer = new StreamWriter(filePath))
        writer.Write(json);
    
}

In [ ]:
WriteToJsonFile("C:\\DCUGoogleDrive\\DataVisualisationAssignmentData\\sourceAndTotalContentType.json", sourceAndTotalContentType);
WriteToJsonFile("C:\\DCUGoogleDrive\\DataVisualisationAssignmentData\\sourceAndMonthAddedCounts.json", sourceAndMonthAddedCounts);
WriteToJsonFile("C:\\DCUGoogleDrive\\DataVisualisationAssignmentData\\splitBySourceAndGenre.json", splitBySourceAndGenre);
WriteToJsonFile("C:\\DCUGoogleDrive\\DataVisualisationAssignmentData\\countryCount.json", countryCount);

In [ ]:
File.WriteAllLines("C:\\DCUGoogleDrive\\DataVisualisationAssignmentData\\sourceAndTotalContentType.csv", sourceAndTotalContentType.Select(x => string.Join(",", x)));
File.WriteAllLines("C:\\DCUGoogleDrive\\DataVisualisationAssignmentData\\sourceAndMonthAddedCounts.csv", sourceAndMonthAddedCounts.Select(x => string.Join(",", x)));
File.WriteAllLines("C:\\DCUGoogleDrive\\DataVisualisationAssignmentData\\splitBySourceAndGenre.csv", splitBySourceAndGenre.Select(x => string.Join(",", x)));
File.WriteAllLines("C:\\DCUGoogleDrive\\DataVisualisationAssignmentData\\countryCount.csv", countryCount.Select(x => string.Join(",", x)));

In [ ]:
splitBySourceAndGenre

index,Source,Genre,Amount
0,netflix,Drama,2219
1,netflix,International,3025
2,netflix,Comedy,1575
3,netflix,Thriller,408
4,netflix,Romance,699
5,netflix,Independent,423
6,netflix,Action,663
7,netflix,Adventure,663
8,netflix,Horror,262
9,netflix,Crime,392


In [ ]:
((decimal)allData.Count(_ => _.Source =="amazon-prime-video")/ allData.Count()) * 100

41.125269554989217800431287980

## Everything below here was exploratory and no longer works!

## Lets get Charting!

In [ ]:
public Color NetflixRed = Color.fromHex("#E50914");
public Color NetflixBlack = Color.fromHex("#000000");
public Color HuluGreen = Color.fromHex("#1ce783");
public Color HuluBlack = Color.fromHex("#040405");
public Color AmazonBlue = Color.fromHex("#05A0D1");
public Color AmazonDarkBlue = Color.fromHex("#232F3E");
public Color AmazonOrange = Color.fromHex("#FF9900");
public Color DisneyBlue = Color.fromHex("#13245E");

In [ ]:
var names = new string[]{"Movie", "Tv Show"};
var sfValues = new  [] {netflix.Count(_ => _.Type == "Movie"), netflix.Count(_ => _.Type =="TV Show")};

var chart1 = Chart.Combine(new []
    {
Chart2D.Chart.Column<string, int, string>(new []{"Movie"}, new  [] {netflix.Count(_ => _.Type == "movie")}, Color: NetflixRed, ShowLegend: false),
Chart2D.Chart.Column<string, int, string>(new []{"TV Show"}, new  [] {netflix.Count(_ => _.Type == "tv-show")}, Color: NetflixBlack, ShowLegend: false),
    });
    chart1.WithTitle("Number of Movies and TV Shows on Netflix Streaming platform")

<!-- Plotly chart will be drawn inside this DIV -->

## Things to do with this

Colour code it, black and red

In [ ]:
public static string DateTimeToYearMonthString(DateTime date){
return $"{date.Year}-{date.Month}";
}

In [ ]:
var movieList = netflix.Where(_ => _.Type == "movie");
var tvShowList = netflix.Where(_ => _.Type == "tv-show");
var moviesGroupedByMonthAdded = movieList.OrderBy(_ => _.DateAdded).Select(_ => new { Date = _.DateAdded, Name = DateTimeToYearMonthString(DateTime.Parse(_.DateAdded.ToString()))}).GroupBy(_ => _.Name);

var tvShowsGroupedByMonthAdded = tvShowList.OrderBy(_ => _.DateAdded).Select(_ => new { Date = _.DateAdded, Name = DateTimeToYearMonthString(DateTime.Parse(_.DateAdded.ToString()))}).GroupBy(_ => _.Name);


In [ ]:
var moviesCountedByMonth = moviesGroupedByMonthAdded.Select(_ => new {Name = _.Key, Count = _.Count()});
var tvShowsCountedByMonth = tvShowsGroupedByMonthAdded.Select(_ => new {Name = _.Key, Count = _.Count()});

In [ ]:
var adklfjn = new Plotly.NET.TraceObjects.Marker();
adklfjn.SetValue("color", "#000000");
var chart2 = Chart.Combine(new []{
    Chart2D.Chart.Line<string, int, string>(moviesCountedByMonth.Select(_ => _.Name), moviesCountedByMonth.Select(_ => _.Count), Color: NetflixRed).WithTraceName("Movies").WithMarker(adklfjn),
    Chart2D.Chart.Line<string, int, string>(tvShowsCountedByMonth.Select(_ => _.Name), tvShowsCountedByMonth.Select(_ => _.Count), Color: NetflixBlack).WithTraceName("TV Shows")
    });
    chart2.WithLegend(false)

<!-- Plotly chart will be drawn inside this DIV -->

## Changes for this graph
Does this indicate a change in strategy of onboarding? From my memory this lines up with when Disney got serious, or do we just have bad data?

But even with this blips, it can be seen that netflix is rapidly increasing the amount of content on its platform. While it may not seem like the best choice for television consumers, tv shows typically run much longer than movies, so there is likely many more hours of tv shows to watch.
The rapid decline in 2020 is likely possibly due to the start of covid, the uncertainty may have postponed releases. 

##Things to do with this graph
Zoom in, don't care about the left really, the data seems to be missing, or was the onboarding really that slow?
Display the ticks of every 3rd month instead of every second year
Fix the legend or think of how to label the data directly


In [ ]:
public static Title ToTitle(this string title){
    var a = new Title(); 
    a.SetValue("Title", title);
    return a; 
}

In [ ]:
var tvMovieTendency = Chart.Combine(new []{
Chart2D.Chart.Point<int, int, string>(x:new []{netflix.Count(_ => _.Type == "movie")}, y:new []{netflix.Count(_ => _.Type == "tv-show")}),
Chart2D.Chart.Point<int, int, string>(x:new []{amazon.Count(_ => _.Type == "movie")}, y:new []{amazon.Count(_ => _.Type == "tv-show")}),
Chart2D.Chart.Point<int, int, string>(x:new []{disney.Count(_ => _.Type == "movie")}, y:new []{disney.Count(_ => _.Type == "tv-show")}),
Chart2D.Chart.Point<int, int, string>(x:new []{hulu.Count(_ => _.Type == "movie")}, y:new []{hulu.Count(_ => _.Type == "tv-show")}),
Chart2D.Chart.Point<int, int, string>(x:new []{hbo.Count(_ => _.Type == "movie")}, y:new []{hbo.Count(_ => _.Type == "tv-show")}),
});

tvMovieTendency


<!-- Plotly chart will be drawn inside this DIV -->